[Chapter 4] Setting a Strong Baseline Forecast

2. generating a strong baseline forecast

In [78]:
import time
import os
from pathlib import Path
from tqdm import tqdm 
tqdm.pandas()

import numpy as np
import pandas as pd

from functools import partial
from statsforecast.core import StatsForecast
from utilsforecast.evaluation import evaluate
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    HistoricAverage,
    WindowAverage,
    SeasonalWindowAverage,
    RandomWalkWithDrift,
    HoltWinters,
    ETS,
    AutoETS,
    AutoARIMA,
    ARIMA,
    AutoTheta,
    DynamicTheta,
    DynamicOptimizedTheta,
    Theta,
    OptimizedTheta,
    TBATS,
    AutoTBATS,
    MSTL

)
from datasetsforecast.losses import *
from src.utils.ts_utils import forecast_bias

from utilsforecast.plotting import plot_series
from src.utils import plotting_utils
from src.utils.plotting_utils import plot_forecast
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

import warnings
warnings.filterwarnings("ignore")

In [39]:
# read train test val split data (with missing values imputed)

train_df = pd.read_parquet("./data/london_smart_meters/preprocessed/selected_blocks_train_missing_imputed.parquet")
train_df = train_df[['LCLid',"timestamp","energy_consumption","frequency"]]
val_df = pd.read_parquet("./data/london_smart_meters/preprocessed/selected_blocks_val_missing_imputed.parquet")
val_df = val_df[['LCLid',"timestamp","energy_consumption","frequency"]]
test_df = pd.read_parquet("./data/london_smart_meters/preprocessed/selected_blocks_test_missing_imputed.parquet")
test_df = test_df[['LCLid',"timestamp","energy_consumption","frequency"]]
freq = train_df.iloc[0]['frequency']

# date range

print("Min train_df Date: " , train_df.timestamp.min())
print("Max train_df Date: " , train_df.timestamp.max())
print("Min val_df Date: " , val_df.timestamp.min())
print("Max val_df Date: " , val_df.timestamp.max())
print("Min test_df Date: " , test_df.timestamp.min())
print("Max test_df Date: " , test_df.timestamp.max())
print("freq: ", freq)

Min train_df Date:  2012-01-01 00:00:00
Max train_df Date:  2013-12-31 23:30:00
Min val_df Date:  2014-01-01 00:00:00
Max val_df Date:  2014-01-31 23:30:00
Min test_df Date:  2014-02-01 00:00:00
Max test_df Date:  2014-02-27 23:30:00
freq:  30min


In [40]:
# pick a single time series from the dataset for illustration

ts_train = train_df.loc[train_df.LCLid=='MAC000193',['LCLid','timestamp','energy_consumption']]
ts_val = val_df.loc[val_df.LCLid=='MAC000193', ['LCLid','timestamp','energy_consumption']]
ts_test = test_df.loc[test_df.LCLid=='MAC000193', ['LCLid','timestamp','energy_consumption']]

baseline forecast

In [41]:
pred_df = pd.concat([ts_train, ts_val])

In [42]:
# NIXTLA looks for 3 columns: id_col (identify unique ts), time_col (timestamp), target_col (NIXTLA to forecast)

def evaluate_performance(ts_train, ts_test, models, metrics, freq, level, id_col, time_col, target_col, h, metric_df=None):
    if metric_df is None:
        metric_df = pd.DataFrame()  # initialize an empty DataFrame if not provided

    results = ts_test.copy()

    # Timing dictionary to store train and predict durations
    timing = {}

    for model in models:
        model_name = model.__class__.__name__
        evaluation = {}  # Reset the evaluation dictionary for each model

        # Start the timer for fitting and prediction
        start_time = time.time()

        # Instantiate StatsForecast class
        sf = StatsForecast(
            models=[model],
            freq=freq,
            n_jobs=-1,
            fallback_model=Naive()
        )

        # Efficiently predict without storing memory
        y_pred = sf.forecast(
            h=h,
            df=ts_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            level=level,
        )

        # Calculating the duration
        duration = time.time() - start_time
        timing[model_name] = duration

        # Merge prediction results to the original dataframe
        results = results.merge(y_pred, how='left', on=[id_col, time_col])

        ids = ts_train[id_col].unique()
        # Calculate metrics
        for id in ids:
            temp_results = results[results[id_col] == id]
            temp_train = ts_train[ts_train[id_col] == id]
            for metric in metrics:
                metric_name = metric.__name__
                if metric_name == 'mase':
                    evaluation[metric_name] = metric(temp_results[target_col].values,
                                                    temp_results[model_name].values,
                                                    temp_train[target_col].values, seasonality=48)
                else:
                    evaluation[metric_name] = metric(temp_results[target_col].values, temp_results[model_name].values)
            evaluation[id_col] = id
            evaluation['Time Elapsed'] = timing[model_name]

            # Prepare and append this model's results to metric_df
            temp_df = pd.DataFrame(evaluation, index=[0])
            temp_df['Model'] = model_name
            metric_df = pd.concat([metric_df, temp_df], ignore_index=True)

    return results, metric_df


Naive Forecast

In [43]:
from statsforecast.models import Naive

# just the last/most recent observation Y_{t-1} in ts
model = Naive()
display_name = ['Naive']
metrics = pd.DataFrame()

results, metrics = evaluate_performance(
    ts_train=ts_train, 
    ts_test=ts_val, 
    models=[model], 
    metrics=[mase, mae, mse, rmse, smape, forecast_bias], 
    freq=freq,
    level=[],  # Ensure this is correct or adjust as necessary
    id_col='LCLid',
    time_col='timestamp',
    target_col='energy_consumption',
    h=len(ts_val),
    metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.58813,74.340254,MAC000193,0.112895,Naive


moving average forecast

In [44]:
from statsforecast.models import WindowAverage

# take the mean of latest n steps as the forecast
# take a moving average over 48 timesteps (i.e. 48*30-min = 1-day)
model = WindowAverage(window_size=48)
display_name = ['WindowAverage']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage


seasonal naive forecast

In [45]:
from statsforecast.models import SeasonalNaive

# take the Y_{t-k} observation, looking back k steps for each forecast and enabling algo to mimic the last seasonality cycle
model = SeasonalNaive(season_length=48*7)
display_name=['SeasonalNaive']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage
2,1.500975,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.104742,SeasonalNaive


exponential smoothing (ETS) - HoltWinters

In [46]:
# ETS use the weighted average where the weights decrease exponentially as we move farther into the history
# f_{t} = a*y_{t-1} + a*(1-a)*y_{t-2} + a*(1-a)^2*y_{t-3}+...
# 0<=a<=1 is the smoothing parameter that decides how fast or slow the weights should decay

from statsforecast.models import HoltWinters

# Holt-Winters (HW) has thee parameters (alpha, beta, gamma) for smoothing (level, trend, seasonality) 
#                   and seasonality period (m) as input parameters.
#                   can choose additive or multiplicative (below is additive version)
# forecast equation: f_{t+h} = l_{t} + h*b_{t} + s_{t+h-m(k+1)}
# level equation: l_{t} = alpha*y_{t} - s_{t-m} + (1 - alpha)*(l_{t-1} + b_{t-1})
# trend euqation: b_{t} = beta*(l_{t} - l_{t-1}) + (1 - beta)*b_{t-1}
# seasonality equation: s_{t} = gamma*(y_{t} - l_{t-1} - b_{t-1}) + (1 - gamma)*s_{t-m}

# error_type='A' refers to additive error ('M' is multiplicative)
model = HoltWinters(error_type='A', season_length=48)
display_name = ['HoltWinters']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage
2,1.500975,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.104742,SeasonalNaive
3,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.254271,HoltWinters


exponential smoothing (ETS) - AutoETS

In [47]:
# the family of ETS can be think about in terms of the trend and seasonal components
#                   - trend can be none/additive/additive damped
#                   - seasonality can be none/additive/multiplicative

from statsforecast.models import AutoETS

# AutoETS will autoamatically choose 
#   - which ETS model is the best option: simple ETS (none, none), double ETS/Holts (additive, none)
#                                         triple ETS/Holt-Winters (additive, additive)
#   - which parameters and error types are best for each ts

model = AutoETS(model = 'AAA',season_length = 48)
display_name = ['AutoETS']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()


,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage
2,1.500975,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.104742,SeasonalNaive
3,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.254271,HoltWinters
4,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.094316,AutoETS


autoregressive integrated moving average (ARIMA)

In [ ]:
# AutoRegression(p) model: y_{t} = c + phi_{1}*y_{t-1} + phi_{2}*y_{t-2} + ... + phi_{p}*y_{t-p} + epsilon_{t}
# MovingAverage(q) model: y_{t} = c + theta_{1}*epsilon_{t-1} + theta_{2}*epsilon_{t-2} + ... + theta_{q} * espsilon_{t-q}
# (c is intercept, epsilon is white noise)

# ARMA(p, q) model: y_{t} = AR(p) + MA(q)

# ARIMA(p, d, q) model: we do the d order of differencing 
#                       then consider the last p terms in an autoregressive manner 
#                       then inlcude the last q moving average terms to come up with the forecast

# seaonal ARIMA(P, D, Q) model: take the last P seasonal lags (P_{1} -> y_{t-m})
#                                D means the order of seasonal differencing, Q is seasonal values of q

from statsforecast.models import ARIMA

model = ARIMA(order=(2, 1, 1), seasonal_order=(1, 1, 1), season_length=48)
display_name = ['ARIMA']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage
2,1.500975,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.104742,SeasonalNaive
3,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.254271,HoltWinters
4,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.094316,AutoETS
5,1.182500,0.198467,0.104090,0.322629,96.552360,21.642244,MAC000193,12.006619,ARIMA


autoregressive integrated moving average (ARIMA) - AutoARIMA

In [50]:
# AutoARIMA() use automatic way of iterating through the different parameters to find the best p, d, q and P, D, Q
#             can be quite slow for long ts, NIXTLA claims to have the fastest and most accurate version

from statsforecast.models import AutoARIMA

model = AutoARIMA(max_p=2, max_d=1, max_q=2, max_P=2, max_D=1, max_Q=2, stepwise=True, season_length=48)
display_name = ['AutoARIMA']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage
2,1.500975,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.104742,SeasonalNaive
3,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.254271,HoltWinters
4,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.094316,AutoETS
5,1.182500,0.198467,0.104090,0.322629,96.552360,21.642244,MAC000193,12.006619,ARIMA
6,1.721933,0.289004,0.194123,0.440594,97.466826,32.930848,MAC000193,3509.640376,AutoARIMA


theta forecast

In [53]:
# deaseaonalization (remove seasonal component from ts) 
# -> theta coeff application (decompose deseasonalized ts into 2 theta lines)
# -> extrapolation of theta lines (linear reg forecast theta line where theta=0, ets forecast theta line where theta=2)
# -> recomposition to combine 2 theta lines as integrating long-term trend and short-term movements
# -> reseasonalize if data was deseasonalized in the begining

from statsforecast.models import Theta

model = Theta(season_length=48, decomposition_type='additive')
display_name = ['Theta']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

# we can see the seasonality pattern is captured but not hitting the peaks


,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage
2,1.500975,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.104742,SeasonalNaive
3,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.254271,HoltWinters
4,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.094316,AutoETS
5,1.182500,0.198467,0.104090,0.322629,96.552360,21.642244,MAC000193,12.006619,ARIMA
6,1.721933,0.289004,0.194123,0.440594,97.466826,32.930848,MAC000193,3509.640376,AutoARIMA
7,1.428410,0.239740,0.167732,0.409551,104.391205,56.923163,MAC000193,177.212935,Theta


Trigonometric seasonality + Box-Cox transformation + ARMA errors + Trend + Seasonal components (TBATS)

In [ ]:
# TBATS can handle complex seasonality (like more than one seasonality or non-integer seasonal period)

# BATS has parameters (omega, phi, p, q, m_1, m_2, ..., m_t) indicating Box-Cox parameters, damping parameter, ARMA(p, q)
# BATS(1, 1, 0, 0, m_{1}) = Holt-Winters Additive Seasonality
# BATS(1, 1, 0, 0, m_{2}) = Holt-Winters Additive DOuble Seasonality
# BATS has the flexibility for multiple seasonality, limited to only integer-based seasonal periods

# TBATS has parameters (omiega, phi, p, q, {m_1, k_1}, ... , {m_T, k_T})
# work with single, complex, and non-integer seasonality (trigonometric seasonality)
# handle non-linear pattern common in real-world ts (box-cox transformation)
# handle autocorrelcation in the residuals (ARMA errors)

# TBATS steps: 
# box-cox transformation (data can be more closely resembles of a normal distribution, can only used with +ve data)
# -> exponentially smoothed trend (use locally-estimated-scatterplot-smoothing LOESS)
# -> seasonal decomposition using fourier series (trigonometric seasonality)
# -> autoregressive moving average (ARMA)
# -> parameter estimation through a likelihood-based approach
#    (with/without box-cox/trend/trend damping/seasonality, use AutoARMA to determine ARMA parameters, combination to minimize AIC)

# TBATS does not allow for exogenous variables
# NIXTLA also offers AutoTBATS

model = TBATS(seasonal_periods=48, use_trend=True, use_damped_trend=True)
display_name = ['TBATS']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage
2,1.500975,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.104742,SeasonalNaive
3,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.254271,HoltWinters
4,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.094316,AutoETS
5,1.182500,0.198467,0.104090,0.322629,96.552360,21.642244,MAC000193,12.006619,ARIMA
6,1.721933,0.289004,0.194123,0.440594,97.466826,32.930848,MAC000193,3509.640376,AutoARIMA
7,1.428410,0.239740,0.167732,0.409551,104.391205,56.923163,MAC000193,177.212935,Theta
8,1.537973,0.258129,0.128138,0.357964,92.680901,3.616123,MAC000193,21.244082,TBATS


Multiple Seasonal-Trend decomposition using LOESS (MSTL)

In [ ]:
# decompose ts into seasonal components
# employ non-seasonal model to forecast the trend and seasonal naive model to predict seasonal component

model = MSTL(season_length=48)
display_name = ['MSTL']

results, metrics = (
    evaluate_performance(
        ts_train, 
        ts_val, 
        models = [model], 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = len(ts_val),
        metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
        )
)

# evaluation metrics
display(metrics)

# plot
fig = plot_forecast(results, forecast_columns=display_name, forecast_display_names=display_name, timestamp_col ='timestamp')
fig.update_xaxes(type="date", range=["2014-01-01", "2014-01-08"])
fig.show()

# among the all, AutoETS and TBATS perform the best

,mase,mae,mse,rmse,smape,forecast_bias,LCLid,Time Elapsed,Model
0,1.819589,0.305395,0.249304,0.499304,113.588130,74.340254,MAC000193,0.112895,Naive
1,1.857295,0.311723,0.182946,0.427722,100.698209,11.509527,MAC000193,0.108711,WindowAverage
2,1.500975,0.251919,0.190827,0.436838,78.738832,13.735403,MAC000193,0.104742,SeasonalNaive
3,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.254271,HoltWinters
4,1.139452,0.191242,0.100917,0.317675,89.170122,10.577896,MAC000193,21.094316,AutoETS
5,1.182500,0.198467,0.104090,0.322629,96.552360,21.642244,MAC000193,12.006619,ARIMA
6,1.721933,0.289004,0.194123,0.440594,97.466826,32.930848,MAC000193,3509.640376,AutoARIMA
7,1.428410,0.239740,0.167732,0.409551,104.391205,56.923163,MAC000193,177.212935,Theta
8,1.537973,0.258129,0.128138,0.357964,92.680901,3.616123,MAC000193,21.244082,TBATS
9,1.506074,0.252775,0.148282,0.385074,117.315590,48.108205,MAC000193,5.374519,MSTL


Validation Set - for AutoETS and TBATS

In [ ]:
# prepare data

ids = list(train_df.LCLid.unique()[:500]) # slicing dataframe for the sake of time.  May want to consider setting this low if working on a slower machine

train_df = train_df[train_df.LCLid.isin(ids)]
val_df = val_df[val_df.LCLid.isin(ids)]
test_df = test_df[test_df.LCLid.isin(ids)]

print("Length of validation Data: ", len(val_df[val_df.LCLid =='MAC000948'].LCLid))

Length of validation Data:  1488


In [ ]:
# input parameters

validation_models =  [AutoETS(model = 'AAA',season_length = 48), 
                      TBATS(seasonal_periods  = 48)]

validation_models_names = [model.__class__.__name__ for model in validation_models]
metric_df = pd.DataFrame([])
h_val = 1488

In [ ]:
# prediction and evaluation metrics

aggval_metrics = pd.DataFrame()

baseline_val_pred_df, aggval_metrics = (
    evaluate_performance(
        train_df[["LCLid","timestamp","energy_consumption"]], 
        val_df[["LCLid","timestamp","energy_consumption"]], 
        models =validation_models, 
        metrics = [mase, mae, mse, rmse, smape,forecast_bias], 
        freq = freq,
        level = [] ,
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        h = h_val,
        metric_df = aggval_metrics
        )
)

In [ ]:
# print autoets results

autoets_val_metric_df = aggval_metrics[aggval_metrics.Model =='AutoETS']
overall_metrics_val_autoets = {
    "Algorithm": "AutoETS",
    "MAE": mae(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values),
    "MSE": mse(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values),
    "meanMASE": autoets_val_metric_df.mase.mean(),
    "Forecast Bias": forecast_bias(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.AutoETS.values)
}
print(overall_metrics_val_autoets)


# print tbats results
 
tbats_val_metric_df = aggval_metrics[aggval_metrics.Model =='TBATS']
overall_metrics_val_tbats = {
    "Algorithm": "TBATS",
    "MAE": mae(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.TBATS.values),
    "MSE": mse(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.TBATS.values),
    "meanMASE": tbats_val_metric_df.mase.mean(),
    "Forecast Bias": forecast_bias(baseline_val_pred_df.energy_consumption.values, baseline_val_pred_df.TBATS.values)
}
print(overall_metrics_val_tbats)

{'Algorithm': 'AutoETS', 'MAE': 0.11972959, 'MSE': 0.057773903, 'meanMASE': 1.0285096, 'Forecast Bias': 4.263771325349808}
{'Algorithm': 'TBATS', 'MAE': 0.27828524, 'MSE': 0.70151895, 'meanMASE': 2.4443083, 'Forecast Bias': -25.927910208702087}


In [ ]:
# comparison summary

baseline_val_metrics_df = pd.DataFrame([overall_metrics_val_autoets, overall_metrics_val_tbats])

display(baseline_val_metrics_df.style.format({"MAE": "{:.3f}", 
                          "MSE": "{:.3f}", 
                          "meanMASE": "{:.3f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen', subset=["MAE","MSE","meanMASE"]))

# autoETS performs better in all 3 metrics

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,AutoETS,0.120,0.058,1.029,4.26%
1,TBATS,0.278,0.702,2.444,-25.93%


In [ ]:
# plot distribution of MASE

fig = px.histogram(aggval_metrics, 
                   x="mase", 
                   color="Model",
                   pattern_shape="Model", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")

fig.show()

In [62]:
# save validation forecast and metrics

os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")
baseline_val_pred_df.to_pickle(output/"baseline_val_prediction_df.pkl")
baseline_val_metrics_df.to_pickle(output/"baseline_val_metrics_df.pkl")
aggval_metrics.to_pickle(output/"baseline_val_aggregate_metrics.pkl")

Test Set - for AutoETS and TBATS

In [63]:
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import rmse as rmse_local
from utilsforecast.losses import mae as mae_local
from utilsforecast.losses import mse as mse_local
from utilsforecast.losses import mase as mase_local

from functools import partial

In [66]:
_train_df = pd.concat([train_df, val_df])
print("Length of test Data: ", len(test_df[test_df.LCLid =='MAC000948'].LCLid))

Length of test Data:  1296


In [67]:
# input parameters

test_models =  [ AutoETS(model = 'AAA',season_length = 48), TBATS(seasonal_periods  = 48)]
h_test = 1296

In [ ]:
# test forecast 

sf = StatsForecast(
    models=test_models,
    freq=freq,
    n_jobs=-1,
    fallback_model= SeasonalNaive(season_length=48)
)

# memory efficient predictions
baseline_test_pred_df = sf.forecast(df=_train_df, 
                                        h=h_test, 
                                        level=[],   
                                        id_col = 'LCLid',
                                        time_col = 'timestamp',
                                        target_col = 'energy_consumption',
        )
baseline_test_pred_df = pd.merge(baseline_test_pred_df, test_df[['timestamp', 'LCLid', 'energy_consumption']], on = ['LCLid','timestamp'], how = 'left')

In [72]:
# evaluation metrics
fcst_mase = partial(mase_local, seasonality=48)

baseline_test_metrics_df = evaluate(baseline_test_pred_df, 
        metrics=[rmse_local, mae_local, mse_local,fcst_mase],  
        train_df = _train_df[['timestamp', 'LCLid', 'energy_consumption']],      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption'
        )

In [73]:
cols_to_exclude  = ['LCLid', 'metric']
test_columns = [col for col in baseline_test_metrics_df.columns if col not in cols_to_exclude]
# test_column = ['AutoETS','TBATS']

baseline_test_metrics_df = pd.melt(baseline_test_metrics_df, id_vars = ['LCLid','metric'],value_vars= test_columns, var_name='Algorithm' , value_name='value')
baseline_test_metrics_df = baseline_test_metrics_df.pivot_table(index = ['LCLid','Algorithm'], columns = 'metric', values = 'value').reset_index()
baseline_test_metrics_df.head(4)

metric,LCLid,Algorithm,mae,mase,mse,rmse
0,MAC000061,AutoETS,0.058050,0.994473,0.006233,0.078950
1,MAC000061,TBATS,0.086410,1.480309,0.011605,0.107724
2,MAC000062,AutoETS,0.091543,1.020324,0.028732,0.169505
3,MAC000062,TBATS,1.167705,13.015079,1.707811,1.306833


In [ ]:
# comparison summary

from datasetsforecast.losses import *

agg_test_metrics = []  # Initialize an empty list to store the metrics dictionaries

for model in validation_models_names: # ['AutoETS', 'ABATS']
    actual_series = baseline_test_pred_df['energy_consumption'].values
    pred_series = baseline_test_pred_df[model].values
    
    # Create a dictionary for the current model's metrics
    agg_test_metrics1 = {
        "Algorithm": model,
        "MAE": mae(actual_series, pred_series),
        "MSE": mse(actual_series, pred_series),
        "meanMASE": baseline_test_metrics_df[baseline_test_metrics_df.Algorithm == model].mase.mean(),
        "Forecast Bias": forecast_bias(actual_series, pred_series),
    }
    
    # Append the dictionary to the list
    agg_test_metrics.append(agg_test_metrics1)

agg_test_metrics_df = pd.DataFrame(agg_test_metrics)
agg_test_metrics_df

,Algorithm,MAE,MSE,meanMASE,Forecast Bias
0,AutoETS,0.119331,0.060437,0.997021,-9.876164
1,TBATS,1.190208,19.400761,9.213356,20.267491


In [75]:
# plot of distribution of MASE

fig = px.histogram(baseline_test_metrics_df, 
                   x="mase", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
#fig = format_plot(fig, xlabel="MASE", ylabel="Probability Density", title="Distribution of MASE in the dataset")
#fig.update_layout(xaxis_range=[0,10])
# fig.write_image("imgs/chapter_4/mase_dist_test.png")
fig.show()

In [76]:
# save test forecast and metrics

os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")
baseline_test_pred_df.to_pickle(output/"baseline_test_prediction_df.pkl")
baseline_test_metrics_df.to_pickle(output/"baseline_test_metrics_df.pkl")
agg_test_metrics_df.to_pickle(output/"baseline_test_aggregate_metrics.pkl")